<a href="https://colab.research.google.com/github/Serag11/Machine-Learning/blob/main/Random_forest_XGBoost_on_spotify_dataset_NTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets
import pandas as pd
import opendatasets as od
od.download('https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset')
df = pd.read_csv('/content/-spotify-tracks-dataset/dataset.csv')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sgejiojoj
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset


100%|██████████| 8.17M/8.17M [00:00<00:00, 907MB/s]

In [3]:
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [4]:
df['artists'].value_counts()

,count
artists,
The Beatles,279
George Jones,271
Stevie Wonder,236
Linkin Park,224
Ella Fitzgerald,222
...,...
Templo Soul;Pregador Luo,1
Purples;Paulo Cesar Baruk,1
Ahmet Kilic;Stoto;Adeba,1


In [5]:
track_genre_counts = df.groupby('track_id')['track_genre'].nunique()
print(track_genre_counts.head(10))

track_id
0000vdREvCVMxbQTkS888c    1
000CC8EParg64OmTxVnZ0p    1
000Iz0K615UepwSJ5z2RE5    1
000RDCYioLteXcutOjeweY    1
000qpdoc97IMTBvF8gwcpy    1
0017XiMkqbTfF2AUOzlhj6    1
001APMDOl3qtx1526T11n1    2
001YQlnDSduXd5LgBd66gT    2
001pyq8FLNSL1C8orNLI0b    1
002qpSULhHAw6DGqFxbaO1    1
Name: track_genre, dtype: int64


In [6]:
tracks_to_exclude = track_genre_counts[track_genre_counts > 1].index
df_filtered = df[~df['track_id'].isin(tracks_to_exclude)]
print(f"Original DataFrame shape: {df.shape}")
print(f"Filtered DataFrame shape: {df_filtered.shape}")
display(df_filtered.head())


Original DataFrame shape: (114000, 21)
Filtered DataFrame shape: (73790, 21)


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.210,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.469,0.000000,0.0829,0.167,119.949,4,acoustic
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,...,-8.700,1,0.0369,0.294,0.000000,0.1510,0.669,130.088,4,acoustic
10,10,4mzP5mHkRvGxdhdGdAH7EJ,Zack Tabudlo,Episode,Give Me Your Forever,74,244800,False,0.627,0.3630,...,-8.127,1,0.0291,0.279,0.000000,0.0928,0.301,99.905,4,acoustic


In [7]:
df_filtered.drop(columns= ['Unnamed: 0' , 'artists' , 'album_name' , 'track_name' , 'duration_ms' , 'explicit' ] , inplace= True)
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73790 entries, 2 to 113999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          73790 non-null  object 
 1   popularity        73790 non-null  int64  
 2   danceability      73790 non-null  float64
 3   energy            73790 non-null  float64
 4   key               73790 non-null  int64  
 5   loudness          73790 non-null  float64
 6   mode              73790 non-null  int64  
 7   speechiness       73790 non-null  float64
 8   acousticness      73790 non-null  float64
 9   instrumentalness  73790 non-null  float64
 10  liveness          73790 non-null  float64
 11  valence           73790 non-null  float64
 12  tempo             73790 non-null  float64
 13  time_signature    73790 non-null  int64  
 14  track_genre       73790 non-null  object 
dtypes: float64(9), int64(4), object(2)
memory usage: 9.0+ MB


/tmp/ipython-input-2700824936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns= ['Unnamed: 0' , 'artists' , 'album_name' , 'track_name' , 'duration_ms' , 'explicit' ] , inplace= True)


In [8]:
df_filtered.drop_duplicates(inplace= True , subset= 'track_id')
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73442 entries, 2 to 113999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          73442 non-null  object 
 1   popularity        73442 non-null  int64  
 2   danceability      73442 non-null  float64
 3   energy            73442 non-null  float64
 4   key               73442 non-null  int64  
 5   loudness          73442 non-null  float64
 6   mode              73442 non-null  int64  
 7   speechiness       73442 non-null  float64
 8   acousticness      73442 non-null  float64
 9   instrumentalness  73442 non-null  float64
 10  liveness          73442 non-null  float64
 11  valence           73442 non-null  float64
 12  tempo             73442 non-null  float64
 13  time_signature    73442 non-null  int64  
 14  track_genre       73442 non-null  object 
dtypes: float64(9), int64(4), object(2)
memory usage: 9.0+ MB


/tmp/ipython-input-1472239611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop_duplicates(inplace= True , subset= 'track_id')


In [9]:
print(f"number of unique values in the original df {df['track_id'].nunique()}")
print(f"number of unique values in the filtered df {df_filtered['track_id'].nunique()}")

number of unique values in the original df 89741
number of unique values in the filtered df 73442


In [10]:
df_filtered.drop(columns= ['track_id'] , inplace= True)
df_filtered.head()

/tmp/ipython-input-682503597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns= ['track_id'] , inplace= True)


,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
2,57,0.438,0.3590,0,-9.734,1,0.0557,0.210,0.000000,0.1170,0.120,76.332,4,acoustic
3,71,0.266,0.0596,0,-18.515,1,0.0363,0.905,0.000071,0.1320,0.143,181.740,3,acoustic
4,82,0.618,0.4430,2,-9.681,1,0.0526,0.469,0.000000,0.0829,0.167,119.949,4,acoustic
8,74,0.625,0.4140,0,-8.700,1,0.0369,0.294,0.000000,0.1510,0.669,130.088,4,acoustic
10,74,0.627,0.3630,8,-8.127,1,0.0291,0.279,0.000000,0.0928,0.301,99.905,4,acoustic


In [11]:
df_filtered.drop(columns= ['time_signature'] , inplace= True)
df_filtered.head()

/tmp/ipython-input-2553210524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns= ['time_signature'] , inplace= True)


,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
2,57,0.438,0.3590,0,-9.734,1,0.0557,0.210,0.000000,0.1170,0.120,76.332,acoustic
3,71,0.266,0.0596,0,-18.515,1,0.0363,0.905,0.000071,0.1320,0.143,181.740,acoustic
4,82,0.618,0.4430,2,-9.681,1,0.0526,0.469,0.000000,0.0829,0.167,119.949,acoustic
8,74,0.625,0.4140,0,-8.700,1,0.0369,0.294,0.000000,0.1510,0.669,130.088,acoustic
10,74,0.627,0.3630,8,-8.127,1,0.0291,0.279,0.000000,0.0928,0.301,99.905,acoustic


In [12]:
df_filtered.drop(columns= [ 'key' , 'mode'] , inplace= True)
df_filtered.head()

/tmp/ipython-input-2275340028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns= [ 'key' , 'mode'] , inplace= True)


,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
2,57,0.438,0.3590,-9.734,0.0557,0.210,0.000000,0.1170,0.120,76.332,acoustic
3,71,0.266,0.0596,-18.515,0.0363,0.905,0.000071,0.1320,0.143,181.740,acoustic
4,82,0.618,0.4430,-9.681,0.0526,0.469,0.000000,0.0829,0.167,119.949,acoustic
8,74,0.625,0.4140,-8.700,0.0369,0.294,0.000000,0.1510,0.669,130.088,acoustic
10,74,0.627,0.3630,-8.127,0.0291,0.279,0.000000,0.0928,0.301,99.905,acoustic


In [13]:
from sklearn.preprocessing import OneHotEncoder , StandardScaler

encoder = OneHotEncoder(sparse_output=False)
encoded_df = pd.DataFrame(encoder.fit_transform(df_filtered[['track_genre']]))

scalar = StandardScaler()
scaled_df = pd.DataFrame(scalar.fit_transform(df_filtered.drop(columns= ['track_genre'])))

y = encoded_df
X = scaled_df

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize the Logistic Regression model for multiclass classification
# 'lbfgs' is a good solver for multiclass problems
# 'multinomial' uses softmax regression for multiclass classification
# Increased max_iter to ensure convergence
model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, random_state=42)

# Convert y_train and y_test from one-hot encoded to label encoded format
y_train_labels = np.argmax(y_train.values, axis=1)
y_test_labels = np.argmax(y_test.values, axis=1)

# Train the model
model.fit(X_train, y_train_labels)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model Accuracy: 0.2535


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the Random Forest Classifier model
# You can tune hyperparameters like n_estimators (number of trees) for better performance
model_rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# Train the model using the label-encoded target variable
model_rf.fit(X_train, y_train_labels)

# Make predictions on the test set
y_pred_rf = model_rf.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test_labels, y_pred_rf)
print(f"Random Forest Model Accuracy: {accuracy_rf:.4f}")

Random Forest Model Accuracy: 0.4493


In [17]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Initialize the XGBoost Classifier model
# You can tune hyperparameters for better performance
model_xgb = XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train_labels)), eval_metric='mlogloss', use_label_encoder=False, random_state=42, n_jobs=-1)

# Train the model using the label-encoded target variable
model_xgb.fit(X_train, y_train_labels)

# Make predictions on the test set
y_pred_xgb = model_xgb.predict(X_test)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test_labels, y_pred_xgb)
print(f"XGBoost Model Accuracy: {accuracy_xgb:.4f}")

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:16:22] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Model Accuracy: 0.4271
